# Versao final - projeto de Redes Sociais

In [1]:
#Import das bibliotecas e criacao do Graph graph dirigido
import pandas as pd
import netpixi #import das bibliotecas necessáriavalue_countss
from netpixi.integration.gt import *
from regression.integration.gt import *
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib

from random import random, seed

graph = Graph(directed=True) #dirigidonão

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [2]:
# Datesets contendo as citacoes e as informacoes das patentes
citacoes = pd.read_csv("cite75_99.txt",sep=",")
patent_data = pd.read_csv("apat63_99.txt",sep=",")

In [3]:
# Como iremos trabalhar com somente patentes da categoria de computacao
# Iremos remover todos que pertencem a outras categorias
computacao = patent_data[patent_data["CAT"]==3] #CAT 2 esta na documentacao como sendo de computacao
patentes_computacao = computacao["PATENT"] #pd.series com o numero de todas as patentes de computacao

In [4]:
# Somente uma fracao dessas patentes sera utilizada para montar a rede
# e fazer a regressao logistica
am_pat_comp = patentes_computacao
am_pat_comp

6          3070807
7          3070808
196        3070997
239        3071040
241        3071042
            ...   
2923714    6009347
2923715    6009348
2923716    6009349
2923717    6009350
2923718    6009351
Name: PATENT, Length: 204199, dtype: int64

In [5]:
# Nesta celula, estamos retirando as patentes gerais que nao pertencem a computacao
cit_comp = citacoes[citacoes["CITING"].isin(am_pat_comp)]
cit_comp = cit_comp[cit_comp["CITED"].isin(am_pat_comp)]

## Preparando o grafo

In [ ]:
# Patentes disruptivas eh a variavel independente e patentes importantes eh a variavel dependente
# patentes disruptivas sao medidas pela quantidade de subcategorias de comp que citam ela (qualitativa)
# patentes importantes sao medidas pelo pageRank (quantitativa)
# independente qualitativa e dependente quantitativa
vertex = []
edge = []

# Funcao responsavel por contruir o grafo a partir das informacoes do dataframe cit_comp
def citting(id1,id2):
    id1 = int(id1)
    id2 = int(id2)
    if not graph.has_vertex(id1):
        vertex.append(id1)
        graph.add_vertex(id1)

    if not graph.has_vertex(id2):
        graph.add_vertex(id2)
        vertex.append(id2)
    if not graph.has_edge(id1,id2):
        edge.append([id1,id2])
        edge.append([id1,id2])#ver jeito melhor
        graph.add_edge(id1,id2)
    return id1,id2

a = cit_comp[['CITING', 'CITED']].apply(lambda x: citting(x['CITING'], x['CITED']), axis=1)

In [ ]:
# Pegando as metricas locais de degree,pageRank e 
n = graph.num_vertices()
m = graph.num_edges()
if graph.is_directed():
    max_edges = n * (n - 1)
else:
    max_edges = n * (n - 1) // 2
d = m / max_edges

dc = gt_total_degree(graph)
dc_in = gt_in_degree(graph)
dc_out = gt_out_degree(graph)

graph.add_vp('degree',dc)
graph.add_vp('in_degree',dc_in)
graph.add_vp('out_degree',dc_out)

print("Número de vértices = ", n)
print("Número de arestas = ", m)
print("Número máximo de arestas = ", max_edges)
print("Densidade = ",d)

In [ ]:
#adicionando coluna de SUBCAT de cada vertice
graph.add_vp('SUBCAT')
for i in graph.all_vertices():

    a = computacao[computacao['PATENT'] == i.get_id()]['SUBCAT']
    teste = a.iloc[0]

    i['SUBCAT'] = str(teste)

In [ ]:
# Adicionando cor correspondente a SUBCAT
graph.add_vp('color')
for v in graph.all_vertices(): 
    if v['SUBCAT'] == "31":
        v['color'] = 0xffd700
    elif v['SUBCAT'] == "32":
        v['color'] = 0x00ff00
    elif v['SUBCAT'] == "33":
        v['color'] = 0xE69138
    elif v['SUBCAT'] == "39":
        v['color'] = 0x6AA84F
    else:
        print("outro")

In [ ]:
# Adicionando a metrica local pageRank para cada vertice
from graph_tool import centrality
pr = centrality.pagerank(graph)
graph.add_vp('pagerank', pr)
gt_data(graph)

In [ ]:
data = gt_data(graph)
info_vertices = data.copy()

In [ ]:
info_vertices.reset_index(inplace=True)

In [ ]:
info_vertices

### Encontrando quantas subcategorias uma patente da subcategoria x e citada 

Para isso, merge entre info_vertices e cit_comp. Depois groupby cited 

In [ ]:
# Merge para pegar subcat de patente CITED e CITTING
cit_comp_info = cit_comp.merge(info_vertices,left_on='CITED',right_on='id',how='left')
cit_comp_info = cit_comp_info.loc[:,['CITING','CITED','SUBCAT','pagerank']]
cit_comp_info = cit_comp_info.rename(columns={'SUBCAT':'SUBCAT_CITED','pagerank':'pagerank_CITED'})

In [ ]:
cit_final = cit_comp_info.merge(info_vertices,left_on='CITING',right_on='id',how='left')
cit_final = cit_final.loc[:,['CITING','CITED','SUBCAT_CITED','pagerank_CITED','SUBCAT','pagerank']]
cit_final = cit_final.rename(columns={'SUBCAT':'SUBCAT_CITING','pagerank':'pagerank_CITING'})

In [ ]:
cit_final = cit_final.loc[:,['CITED','SUBCAT_CITED','pagerank_CITED','SUBCAT_CITING']]

In [ ]:
cit_final

In [ ]:
cit_final_entre_pat_diferentes = cit_final[cit_final['SUBCAT_CITED'] != cit_final['SUBCAT_CITING']]
cit_final_entre_pat_iguais = cit_final[cit_final['SUBCAT_CITED'] == cit_final['SUBCAT_CITING']]

In [ ]:
a = cit_final_entre_pat_diferentes.groupby(by=['CITED']).nunique()
b = cit_final_entre_pat_iguais.groupby(by=['CITED']).nunique()

In [ ]:
a = a.reset_index() #Patente e por quantas subcategorias diferentes ela 'e citada'
b = b.reset_index() #Patente so esta aqui caso tenha sido citada por uma patente de mesma categoria pelo menos uma vez

In [ ]:
a

In [ ]:
b

Caso patente esteja somente em b,quant_subcat=0.

Caso patente esteja em a e b,quant_subcat=valor em a

Caso patente esteja somente em a,quant_subcat=valor em a

In [ ]:
a = a.loc[:,['CITED','SUBCAT_CITING']]
b = b.loc[:,['CITED','SUBCAT_CITING']]
a = a.rename(columns={'SUBCAT_CITING':'quant_subcat_diferentes'})
b = b.rename(columns={'SUBCAT_CITING':'quant_subcat_igual'})

In [ ]:
correct = a.merge(b,how='outer') #Todas as patentes
# correct['coluna1'].fillna(0, inplace=True)
#Se diferentes e igual for null => quant_total = 0
#Se diferentes for Nan e igual for 1 => quant_total = 0
#Se diferentes for 1 e igual for Nan => quant_total = 1
#Se diferentes for 1 e igual for 1 => quant_total = 1
#...
correct['quant_subcat_diferentes'].fillna(0,inplace=True)
correct['quant_subcat_igual'].fillna(1,inplace=True)
correct['quant_subcat_total'] = correct['quant_subcat_diferentes'] + correct['quant_subcat_igual'] - 1

In [ ]:
correct['quant_subcat_total'].describe()

In [ ]:
# Dataframe correct possui todas as patentes de computacao que foram citadas
# Fazer agora um merge entre correct e info_vertices para pegar o pageRank

final = correct.merge(info_vertices,left_on="CITED",right_on="id")
final

In [ ]:
# Fazendo o staircase encoding para fazer a regressao logistica
values = list(final['quant_subcat_total'].unique())
values.sort()

def staircase_encoding(vals,df):
    length = len(vals) - 1

    dummies = [[] for i in range(length)]

    for value in df['quant_subcat_total']:
        index = vals.index(value)

        for i in range(index):
            dummies[i].append(1)

        for i in range(index, length):
            dummies[i].append(0)

    for value, dummy in zip(values[1:], dummies):
        df[f'quant_subcat_total{value}'] = dummy
    
    return df

data_reg = staircase_encoding(values,final)

In [ ]:
dataa = data_reg.loc[:,['CITED','quant_subcat_total','SUBCAT','pagerank','quant_subcat_total1.0','quant_subcat_total2.0','quant_subcat_total3.0']]

In [ ]:
dataa = dataa.rename(columns={'quant_subcat_total1.0':'quant_subcat_total1','quant_subcat_total2.0':'quant_subcat_total2','quant_subcat_total3.0':'quant_subcat_total3'})

## Regressao Logistica

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import regression as reg

from scipy import stats
from matplotlib import pyplot as plt

In [ ]:
result = reg.linear(data=dataa, formula='np.log(pagerank) ~ quant_subcat_total1 + quant_subcat_total2 + quant_subcat_total3')

result.nano_summary()

In [ ]:
result.plot_residuals()

In [ ]:
sns.boxplot(x=dataa['quant_subcat_total'], y=dataa['pagerank'], linewidth=1)
# sns.swarmplot(x=dataa['quant_subcat_total1'], y=dataa['pagerank'], linewidth=1)